<a href="https://colab.research.google.com/github/josephxlp/PyTorch100Days/blob/main/W3DAY16_PyTorchLightning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [10]:
!pip install pytorch-lightning --q

In [11]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
import pytorch_lightning as pl

In [12]:
transform = transforms.ToTensor()
train_ds = datasets.MNIST(root='./data', train=True, download=True, transform=transform)
test_ds = datasets.MNIST(root='./data', train=False, download=True, transform=transform)

In [13]:
train_dl = DataLoader(train_ds, batch_size=128, shuffle=True)
valid_dl = DataLoader(test_ds, batch_size=128, shuffle=False)

In [14]:
class MNIST_Classifier(pl.LightningModule):
    def __init__(self):
        super().__init__()
        self.fc1 = nn.Linear(28*28, 128)
        self.fc2 = nn.Linear(128, 10)

    def forward(self, x):
        x = x.view(-1, 28*28)
        x = F.relu(self.fc1(x))
        return self.fc2(x)

    def training_step(self, batch, batch_idx):
        images, labels = batch
        logists = self(images)
        loss = F.cross_entropy(logists, labels)
        self.log('train_loss', loss)
        return loss

    def validation_step(self, batch, batch_idx):
        images, labels = batch
        logists = self(images)
        loss = F.cross_entropy(logists, labels)
        acc = (logists.argmax(dim=1) == labels).float().mean()
        self.log('valid_loss', loss)
        self.log('valid_acc', acc)
       # return loss

    def configure_optimizers(self):
        return torch.optim.Adam(self.parameters(), lr=1e-3)


In [ ]:
model = MNIST_Classifier()
trainer = pl.Trainer(max_epochs=10, log_every_n_steps=50)
trainer.fit(model, train_dl, valid_dl)

In [18]:
# Access the validation and training losses
train_losses = trainer.callback_metrics['train_loss'].cpu().numpy()
valid_losses = trainer.callback_metrics['valid_loss'].cpu().numpy()
valid_acc = trainer.callback_metrics['valid_acc'].cpu().numpy()
print(f"Final Train Loss: {train_losses}")
print(f"Final Validation Loss: {valid_losses}")
print(f"Final Validation Accuracy: {valid_acc}")

# Getting predictions on the test set
model.eval() # Set the model to evaluation mode
predictions = []
true_labels = []

with torch.no_grad(): # Disable gradient calculation
    for images, labels in valid_dl: # Use the validation dataloader for prediction
        logists = model(images)
        preds = logists.argmax(dim=1)
        predictions.extend(preds.cpu().numpy())
        true_labels.extend(labels.cpu().numpy())

print("\nSample Predictions:")
print(predictions[:10])
print("Sample True Labels:")
print(true_labels[:10])

Final Train Loss: 0.10250356793403625
Final Validation Loss: 0.07516660541296005
Final Validation Accuracy: 0.977400004863739

Sample Predictions:
[np.int64(7), np.int64(2), np.int64(1), np.int64(0), np.int64(4), np.int64(1), np.int64(4), np.int64(9), np.int64(5), np.int64(9)]
Sample True Labels:
[np.int64(7), np.int64(2), np.int64(1), np.int64(0), np.int64(4), np.int64(1), np.int64(4), np.int64(9), np.int64(5), np.int64(9)]


In [17]:
#!jupyter nbconvert --clear-output --inplace W3DAY16_PyTorchLightning.ipynb